In [204]:
import pandas as pd
import numpy as np

df_h=pd.read_excel("Hydrogen_transition_probabilities_v1.xls")
df_h=df_h.query("upper_level<=20" )


In [206]:
recompute_cumulative_transition_prob=False
if recompute_cumulative_transition_prob:
    df_h=df_h.sort_values(['upper_level','lower_level'])

    df_h['TransProbEsp']=df_h['TransProbEsp'].astype(str).str.replace('−','-').astype(float)

    df_h['CumulativeTransProbEsp']=df_h.groupby('upper_level')['TransProbEsp'].cumsum()/df_h.groupby('upper_level')['TransProbEsp'].transform(sum)

    df_h['CumulativeTransProbEsp_low']=df_h.groupby('upper_level')['CumulativeTransProbEsp'].shift(1).replace(np.nan,0)
    

In [220]:
def gen_transition(n0,df_h):
    r=np.random.rand()
    df_h_transition=df_h[df_h.upper_level==n0]
    w_r=df_h_transition.apply(lambda x: r>x['CumulativeTransProbEsp_low'] and r<=x['CumulativeTransProbEsp'],axis=1)
    #print(list(df_h_transition.loc[w_r,['lower_level','Transition']].iloc[0].values))
    nf,transition=list(df_h_transition.loc[w_r,['lower_level','Transition']].iloc[0].values)
    #print(nf)
    return nf,transition
def gen_transition_sequence(n0,df_h):
    nf=n0
    sequence=[]
    while nf!=1:
        nf,transition=gen_transition(nf,df_h)
        sequence.append(transition)
    return sequence
def get_sequence_energies(sequence,df_h):
    return list(df_h.set_index('Transition').loc[sequence,'dE'].values)
def get_sequence_lifetimes(sequence,df_h):
    return list(1.0/df_h.set_index('Transition').loc[sequence,'TransProbEsp'].values)

In [232]:
n0=20
sequence=gen_transition_sequence(n0,df_h)
sequence_energies=get_sequence_energies(sequence,df_h)
sequence_lifetimes=get_sequence_lifetimes(sequence,df_h)

In [233]:
print(f"sequence={sequence}\n Energies={sequence_energies}\n Lifetimes{sequence_lifetimes}")

sequence=['6–20', '5–6', '3–5', '1–3']
 Energies=[0.34377777777777774, 0.16622222222222227, 0.967111111111111, 12.088888888888889]
 Lifetimes[118709.86122817222, 97.52291788570315, 45.43802253725918, 1.7936897992861116]


In [ ]:
#Lifetimes are too disimilar, longtimes between transitions. 